# Import Libraries

In [ ]:
!pip3 install git+git://github.com/tr1ten/textgenrnn.git@SaveToPath
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

  Cloning git://github.com/tr1ten/textgenrnn.git (to revision SaveToPath) to /tmp/pip-req-build-qb1w0lxt
  Running command git clone -q git://github.com/tr1ten/textgenrnn.git /tmp/pip-req-build-qb1w0lxt
  Running command git checkout -b SaveToPath --track origin/SaveToPath
  Switched to a new branch 'SaveToPath'
  Branch 'SaveToPath' set up to track remote branch 'SaveToPath' from 'origin'.
  Created wheel for textgenrnn: filename=textgenrnn-2.0.0-py3-none-any.whl size=1734487 sha256=ca4340767f41ecfbd04b57aec32dfab9d21379168488881be57f3b613f2ca45a
  Stored in directory: /tmp/pip-ephem-wheel-cache-0oec55wd/wheels/ae/ba/64/7f87c098f2fe9c2c0b722c21569817a3c9734b258c7459c653
Successfully built textgenrnn


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Model Configuration

In [ ]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 8,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
    'save_to':'/content/drive/MyDrive/colab-data/checkpoints/nlp',
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 50,   # set higher to train the model for longer
    'gen_epochs': 1,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

# Train

In [ ]:
file_name = "final-names.txt"
model_name = 'lstm-allname'   # change to set file name of resulting trained models/texts

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'],
    save_to=model_cfg['save_to']
    )

117,949 texts collected.
Training new model w/ 3-layer, 128-cell LSTMs
Training on 720,109 character sequences.
Epoch 1/50
703/703 [==============================] - 451s 634ms/step - loss: 2.2461
####################
Temperature: 0.2
####################
mariana

jaylin

jayleen

####################
Temperature: 0.5
####################
shariah

pherabelle

sharte

####################
Temperature: 1.0
####################
cado

mavel

rosilia

Epoch 2/50
703/703 [==============================] - 438s 623ms/step - loss: 2.0860
####################
Temperature: 0.2
####################
tanisha

aleisha

shanita

####################
Temperature: 0.5
####################
marisha

charmille

shalunda

####################
Temperature: 1.0
####################
jimmir

sulairah

selemai

Epoch 3/50
703/703 [==============================] - 447s 636ms/step - loss: 2.0326
####################
Temperature: 0.2
####################
kashandra

arianna

shantasha

####################
Tempera

You can download a large amount of generated text from your model with the cell below! Rerun the cell as many times as you want for even more text!

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = 's'   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 10
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

100%|██████████| 10/10 [00:02<00:00,  4.01it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You can download the weights and configuration files in the cell below, allowing you recreate the model on your own computer!

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

In [ ]:
textgen.generate(n=2,return_as_list=True,prefix='anm')

100%|██████████| 2/2 [00:00<00:00,  6.53it/s]


['anmol', 'anmol']

# Note
all the preprocessing and post processing stuff is also handled by the textgenrnn to save time.  ( lazy me )